# セットアップ

In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# APIキーを読み込む
load_dotenv(verbose=True)
my_apikey = os.environ.get("MY_OPENAI_KEY")

# chatGPTに質問を投げる

In [ ]:
import openai
from IPython.display import Markdown

# APIキーの設定
openai.api_key = my_apikey

def chat(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        # model="gpt-4", # N/A on my dashboard yet
        messages=messages,
    )
    return response["choices"][0]["message"]["content"]

# 初期セットアップ
initial_prompt = "あなたは私の有能なアシスタントです。私からのあらゆる質問に回答して、より良いアイデアを出す助けになってください。"
conversation_history = [{"role": "system", "content": initial_prompt}]

In [ ]:
# 基本的なメッセージのやり取り
message = "ここに質問を入れてセルを実行"
conversation_history.append({"role": "user", "content": message})
response = chat(conversation_history)
display(Markdown(response))
conversation_history.append({"role": "assistant", "content": response})

# YouTubeチャンネルの動画一覧をAPIで取得して要約させる

In [ ]:
import subprocess
from time import sleep
import json
import pandas as pd
from pathlib import Path

columns = ['publishedAt', 'channelId', 'channelTitle', 'title', 'url']
def extract_data(row):
    """データ抽出処理"""
    video_id = row.id.get('videoId', 'not_a_video')
    snippet = row.snippet
    url = f'https://www.youtube.com/watch?v={video_id}'
    result =[]
    for col in columns:
        if col != 'url':
            result.append(snippet[col])
        else:
            result.append(url)
    return [snippet['publishedAt'], snippet['channelId'], snippet['channelTitle'], snippet['title'], url]


# 指定のチャンネルから動画URLをJSON形式で取得して保存
youtube_apikey = os.environ.get('MY_YOUTUBE_APIKEY')
channelId = os.environ.get('TARGET_CHANNEL_ID')
fetch_result_num = 50 # 最大50件

dataset_dir = Path(channelId)
dataset_dir.mkdir(exist_ok=True)
page_num = 1

base = f'https://www.googleapis.com/youtube/v3/search?key={youtube_apikey}&channelId={channelId}&part=snippet,id&order=date&maxResults={fetch_result_num}'
next_page_token = None
df_list = []
while True:
    # 初回はトークンなし
    if next_page_token:
        curl_detail = base + f'&pageToken={next_page_token}'
    else:
        curl_detail = base
    curl_command = f'curl "{curl_detail}"'

    # APIから動画データ取得・抽出
    result = subprocess.run(curl_command, shell=True, capture_output=True, text=True)
    _json = json.loads(result.stdout)
    next_page_token = _json.get('nextPageToken', None)
    df = pd.DataFrame(_json['items'])
    df['data'] = df.apply(extract_data, axis=1)
    df_page = pd.DataFrame(df.data.tolist(), columns=columns)
    df_list.append(df_page)

    # 2ページ目以降へのトークンがなければ終了
    if not next_page_token:
        break
    sleep(5)

df_result = pd.concat(df_list).sort_values("publishedAt", ascending=True, ignore_index=True)
print(df_result)
df_result.head()

In [ ]:
df_result = pd.concat(df_list).sort_values("publishedAt", ascending=True, ignore_index=True)
df_result = df_result[~df_result.url.str.contains('not_a_video')]
df_result.head()

In [ ]:
from datetime import datetime

df_result.sort_values("publishedAt", ascending=True, inplace=True, ignore_index=True)
channel_name = df_result.channelTitle.values[0]
today = datetime.today().strftime('%Y%m%d')
df_result.to_csv(f'{channel_name}_{today}.csv', index=False)

In [ ]:
# GPT-4のAPIが非公開なので動作しない

# # YouTube動画のタイトル・URL・要約をCSVにまとめる
# base_prompt = '''以下のYoutube動画の字幕データを日本語でかつ全体の内容が欠損しないよう要約してください。
# チャンク毎に要約してください。
# {}
# '''
# for url in df_result.url:
#     conversation_history.append({"role": "user", "content": base_prompt.format(url)})
#     response = chat(conversation_history)
#     display(Markdown(response))
#     conversation_history.append({"role": "assistant", "content": response})

In [ ]:
# 保存済みデータ読込み
csv_path = ''
df_result = pd.read_csv(csv_path)

In [ ]:
# GPT-4にプロンプトを手動で投げる
prompt = '''
これからYouTube動画のURLを渡すので、その字幕データを日本語でかつ全体の内容が欠損しないよう要約してください。
チャンク毎に要約してください。
'''

In [ ]:
# YouTube動画のURLを示して要約させる
start = 15
df_result.iloc[df_result.index>start, :].head(15)[['title', 'url']]

In [ ]:
print(df_result.iloc[df_result.index>start, :].title.head(15).to_string(index=False))